In [1]:
# ============================================================
# 013 Startup Databook
# ============================================================
# This notebook provides a reproducible pipeline to discover and profile startups
# using only publicly available information.
#
# It is designed to:
#   1) Define a set of country-specific PR / startup-media sources (e.g., PR portals,
#      startup news sites) as entry points for discovery.
#   2) Allow users to select a target country and year via widgets.
#   3) Run lightweight discovery using Google Programmable Search (CSE) to collect
#      candidate URLs related to funding announcements and startup activity.
#   4) Respect website access policies by performing a light robots.txt / policy check
#      and only fetching content from URLs deemed safe to crawl.
#   5) Use the OpenAI API to:
#        - classify whether a company should be considered a "startup" (Yes/No),
#        - extract and standardize key company and funding attributes when Yes.
#   6) Deduplicate / reconcile entities (company name normalization and record merging).
#   7) Export a structured dataset (CSV/Parquet/SQLite) suitable for:
#        - investment / BD long-listing,
#        - meeting preparation,
#        - ecosystem mapping and year-over-year trend analysis.
#
# Key principles:
#   - Public information only (no private data sources).
#   - Minimal crawling: fetch only what is necessary, rate-limited, and cached.
#   - Transparency: keep source URLs and confidence signals for auditability.
#   - Reproducibility: parameters, intermediate outputs, and final outputs are all persisted.


In [2]:
# ============================================================
# 013-2 Definition of PR / Startup Media Sources by Country
# ============================================================
# This section defines country-specific PR portals and startup-focused media
# that serve as discovery entry points for startup-related announcements.
#
# These sources are used to identify candidate URLs via Google Programmable
# Search (CSE). They are not intended to represent exhaustive or authoritative
# datasets.
#
# Design principles:
#   - Use broad, inclusive keywords to maximize recall at the discovery stage.
#   - Avoid constraining searches by funding stage (e.g., Series A/B), as
#     terminology varies significantly across countries and regions.
#   - Defer precise classification (funding round, amounts, investors) to
#     later stages using LLM-based extraction.
#   - Keep source definitions declarative and easy to extend.
#
# Each country entry contains:
#   - pr_sites: Press-release distribution platforms (if applicable).
#   - startup_media_sites: Media outlets that frequently cover startups,
#     funding events, and new business launches.
#   - discovery_keywords: Broad keywords related to startup activity and
#     investment, used to construct CSE queries.
#
# ------------------------------------------------------------

COUNTRY_SOURCE_DEFINITIONS = {
    "JP": {
        "country_name": "Japan",
        "pr_sites": [
            "prtimes.jp"
        ],
        "startup_media_sites": [
            "jp.techcrunch.com",
            "thebridge.jp"
        ],
        "discovery_keywords": [
            "資金調達",
            "出資",
            "スタートアップ",
            "新規事業"
        ]
    },

    "US": {
        "country_name": "United States",
        "pr_sites": [
            "prnewswire.com",
            "businesswire.com"
        ],
        "startup_media_sites": [
            "techcrunch.com",
            "venturebeat.com"
        ],
        "discovery_keywords": [
            "funding",
            "investment",
            "startup",
            "company launch"
        ]
    },

    "UK": {
        "country_name": "United Kingdom",
        "pr_sites": [
            "prnewswire.co.uk"
        ],
        "startup_media_sites": [
            "sifted.eu",
            "tech.eu"
        ],
        "discovery_keywords": [
            "funding",
            "investment",
            "startup"
        ]
    },

    "DE": {
        "country_name": "Germany",
        "pr_sites": [
            "presseportal.de"
        ],
        "startup_media_sites": [
            "gruenderszene.de",
            "eu-startups.com"
        ],
        "discovery_keywords": [
            "Finanzierung",
            "Investition",
            "Startup"
        ]
    },

    "FR": {
        "country_name": "France",
        "pr_sites": [
            "prnewswire.com"
        ],
        "startup_media_sites": [
            "frenchweb.fr"
        ],
        "discovery_keywords": [
            "levée de fonds",
            "investissement",
            "startup"
        ]
    },

    "CN": {
        "country_name": "China",
        "pr_sites": [],
        "startup_media_sites": [
            "36kr.com"
        ],
        "discovery_keywords": [
            "融资",
            "投资",
            "创业"
        ]
    },

    "IN": {
        "country_name": "India",
        "pr_sites": [],
        "startup_media_sites": [
            "yourstory.com",
            "inc42.com"
        ],
        "discovery_keywords": [
            "funding",
            "investment",
            "startup"
        ]
    },

    "SG": {
        "country_name": "Singapore",
        "pr_sites": [],
        "startup_media_sites": [
            "e27.co",
            "techinasia.com"
        ],
        "discovery_keywords": [
            "funding",
            "investment",
            "startup"
        ]
    },

    "IL": {
        "country_name": "Israel",
        "pr_sites": [],
        "startup_media_sites": [
            "calcalistech.com"
        ],
        "discovery_keywords": [
            "funding",
            "investment",
            "startup"
        ]
    }
}

# These source definitions are consumed by later steps to dynamically
# generate country- and year-specific CSE queries while keeping the
# discovery stage broad and recall-oriented.


In [3]:
# ============================================================
# 013-3 Input UI (ipywidgets)
# ============================================================
# This section defines the interactive UI for selecting:
#   - Target country (one or multiple)
#   - Target period with year (2010–2030) and month (Jan–Dec)
#
# Default period:
#   - Start: Jan 2025
#   - End:   Feb 2025
#
# The selected parameters are consolidated into a single RUN_CONFIG
# dictionary that is reused throughout the notebook.

import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
MONTH_NAMES = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
MONTH_TO_NUM = {m: i + 1 for i, m in enumerate(MONTH_NAMES)}

def _validate_period(sy, sm, ey, em):
    return (ey, MONTH_TO_NUM[em]) >= (sy, MONTH_TO_NUM[sm])

def _expand_months(sy, sm, ey, em):
    out = []
    y, m = sy, MONTH_TO_NUM[sm]
    ey_num = MONTH_TO_NUM[em]
    while (y < ey) or (y == ey and m <= ey_num):
        out.append(f"{y:04d}-{m:02d}")
        m += 1
        if m == 13:
            m = 1
            y += 1
    return out

# ------------------------------------------------------------
# Widget definitions
# ------------------------------------------------------------
YEAR_OPTIONS = list(range(2010, 2031))

country_options = [(f"{v['country_name']} ({k})", k)
                   for k, v in COUNTRY_SOURCE_DEFINITIONS.items()]

country_select = widgets.SelectMultiple(
    options=country_options,
    value=("JP",),
    description="Country",
    rows=min(10, len(country_options))
)

start_year = widgets.Dropdown(
    options=YEAR_OPTIONS,
    value=2025,
    description="Start Year"
)

start_month = widgets.Dropdown(
    options=MONTH_NAMES,
    value="Jan",
    description="Start Month"
)

end_year = widgets.Dropdown(
    options=YEAR_OPTIONS,
    value=2025,
    description="End Year"
)

end_month = widgets.Dropdown(
    options=MONTH_NAMES,
    value="Feb",
    description="End Month"
)

run_button = widgets.Button(
    description="Set Parameters",
    button_style="primary",
    icon="check"
)

status = widgets.HTML(value="")
output = widgets.Output()

# ------------------------------------------------------------
# State container
# ------------------------------------------------------------
RUN_CONFIG = {}

def _on_run_clicked(_):
    global RUN_CONFIG
    with output:
        clear_output()

        countries = list(country_select.value)
        sy, sm = start_year.value, start_month.value
        ey, em = end_year.value, end_month.value

        if not countries:
            status.value = "<b style='color:#b00020'>Please select at least one country.</b>"
            return

        if not _validate_period(sy, sm, ey, em):
            status.value = "<b style='color:#b00020'>Invalid period: end must be later than or equal to start.</b>"
            return

        months = _expand_months(sy, sm, ey, em)

        RUN_CONFIG = {
            "countries": countries,
            "start_year": sy,
            "start_month": sm,
            "end_year": ey,
            "end_month": em,
            "months": months
        }

        status.value = "<b style='color:#0b6'>Parameters set successfully.</b>"

        print("RUN_CONFIG:")
        for k, v in RUN_CONFIG.items():
            if k == "months":
                print(f"  {k}: {v[0]} ... {v[-1]} (n={len(v)})")
            else:
                print(f"  {k}: {v}")

run_button.on_click(_on_run_clicked)

# ------------------------------------------------------------
# Layout
# ------------------------------------------------------------
ui = widgets.VBox([
    widgets.HTML("<h4>Pipeline Parameters</h4>"),
    widgets.HBox([
        country_select,
        widgets.VBox([
            start_year, start_month,
            end_year, end_month,
            run_button,
            status
        ])
    ]),
    output
])

display(ui)


In [7]:
# ============================================================
# 013-4 OpenAI & Google CSE API Setup and Lightweight Discovery
# ============================================================
# This section defines external API clients and implements a lightweight
# discovery layer using Google Programmable Search (CSE).
#
# Purpose:
#   - OpenAI API:
#       Used later for startup classification and structured information
#       extraction from article text.
#   - Google CSE:
#       Used here for *lightweight discovery only* — identifying candidate
#       URLs from PR portals and startup media without crawling sites directly.
#
# Design principles:
#   - Centralize API configuration in one place.
#   - Fail fast if credentials are missing.
#   - Keep CSE usage minimal (URL, title, snippet only).
#   - Defer crawling and heavy processing to later steps.
#
# ------------------------------------------------------------

# ----------------------------
# 4-0. Load API Keys
# ----------------------------
import os
from dotenv import load_dotenv

# Load env.txt explicitly
load_dotenv("env.txt")

# --- OpenAI ---
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY could not be loaded from env.txt.")
else:
    print("✅ OPENAI_API_KEY loaded successfully")

client = OpenAI(api_key=OPENAI_API_KEY)

# --- Google CSE ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_CX = os.getenv("GOOGLE_CSE_CX")

if GOOGLE_API_KEY is None or GOOGLE_CSE_CX is None:
    raise ValueError("GOOGLE_API_KEY or GOOGLE_CSE_CX is missing in env.txt")
else:
    print("✅ Google CSE credentials loaded successfully")

# ----------------------------
# 4-1. Google CSE Helper Functions
# ----------------------------
# The functions below wrap Google CSE calls and return a normalized
# list of search results.
#
# Only metadata is retrieved at this stage:
#   - URL
#   - Title
#   - Snippet
#   - Display link
#
# No page content is fetched here.

import requests
from typing import List, Dict

def run_cse_search(
    query: str,
    num_results: int = 5,
    start_index: int = 1
) -> List[Dict]:
    """
    Execute a single Google CSE query.

    Parameters
    ----------
    query : str
        Search query string.
    num_results : int
        Number of results to return (max 10 per request).
    start_index : int
        Start index for pagination (1-based).

    Returns
    -------
    List[Dict]
        List of search result metadata dictionaries.
    """
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_CX,
        "q": query,
        "num": min(num_results, 10),
        "start": start_index
    }

    resp = requests.get(url, params=params, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    items = data.get("items", [])
    results = []
    for item in items:
        results.append({
            "query": query,
            "title": item.get("title"),
            "snippet": item.get("snippet"),
            "url": item.get("link"),
            "display_link": item.get("displayLink")
        })

    return results

# ----------------------------
# 4-2. Query Construction Logic
# ----------------------------
# This helper generates country-, site-, keyword-, and month-specific
# queries using:
#   - COUNTRY_SOURCE_DEFINITIONS
#   - RUN_CONFIG (selected countries and months)
#
# Example generated query:
#   site:prtimes.jp 資金調達 2025-01

def build_cse_queries(country_code: str, year_month: str) -> List[str]:
    """
    Build a list of CSE queries for a given country and month.

    Parameters
    ----------
    country_code : str
        ISO-like country code (e.g., 'JP', 'US').
    year_month : str
        Target month in 'YYYY-MM' format.

    Returns
    -------
    List[str]
        List of query strings.
    """
    cfg = COUNTRY_SOURCE_DEFINITIONS[country_code]
    sites = cfg["pr_sites"] + cfg["startup_media_sites"]
    keywords = cfg["discovery_keywords"]

    queries = []
    for site in sites:
        for kw in keywords:
            queries.append(f"site:{site} {kw} {year_month}")

    return queries

# ----------------------------
# 4-3. Lightweight Discovery Runner
# ----------------------------
# This function runs CSE searches for all selected countries and months
# and aggregates raw candidate URLs.
#
# IMPORTANT:
#   - This step intentionally favors recall over precision.
#   - Deduplication and filtering are handled in later steps.

def discover_candidate_urls(run_config: dict, max_results_per_query: int = 5):
    """
    Run Google CSE discovery for all selected countries and months.

    Parameters
    ----------
    run_config : dict
        RUN_CONFIG generated by the input UI.
    max_results_per_query : int
        Number of CSE results to fetch per query.

    Returns
    -------
    List[Dict]
        Raw list of candidate URL records.
    """
    all_results = []

    for country in run_config["countries"]:
        for ym in run_config["months"]:
            queries = build_cse_queries(country, ym)
            for q in queries:
                try:
                    results = run_cse_search(q, num_results=max_results_per_query)
                    for r in results:
                        r["country"] = country
                        r["year_month"] = ym
                        all_results.append(r)
                except Exception as e:
                    print(f"⚠️ CSE query failed: {q} | {e}")

    return all_results

# At this point, the output of discover_candidate_urls(...)
# represents a raw, recall-oriented set of candidate URLs.
# Subsequent sections will handle:
#   - URL normalization and filtering
#   - robots.txt / policy checks
#   - content fetching
#   - startup classification and structuring


✅ OPENAI_API_KEY loaded successfully
✅ Google CSE credentials loaded successfully


In [8]:
# ============================================================
# 013-5 Pre-processing of Candidate URLs
# ============================================================
# This section cleans and normalizes the raw Google CSE outputs and prepares a
# high-quality candidate URL table for downstream steps.
#
# Goals:
#   1) Normalize URLs (remove tracking parameters, unify canonical forms).
#   2) Drop obvious noise (non-http(s), missing URLs, unwanted file types).
#   3) Deduplicate aggressively (same URL returned by multiple queries).
#   4) Attach lightweight metadata (country, year_month, source_domain).
#
# Optional (if supported / desired):
#   - Time filtering at the CSE stage can be approximated using query modifiers.
#     Google CSE does not provide a universal "date range" parameter in the API,
#     but two practical approaches are:
#       (a) Add query operators such as `after:` / `before:` (works variably).
#       (b) Add explicit month tokens (we already do: YYYY-MM) to bias results.
#   - Therefore, this notebook uses:
#       - month tokens (YYYY-MM) in queries for recall
#       - post-filtering heuristics here to enforce the selected period.
#
# Output:
#   - candidate_urls_df: normalized, deduplicated URL candidates
#
# ------------------------------------------------------------

import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import pandas as pd

# ----------------------------
# 5-1. Utilities
# ----------------------------
TRACKING_PARAMS = {
    "utm_source", "utm_medium", "utm_campaign", "utm_term", "utm_content",
    "fbclid", "gclid", "yclid", "mc_cid", "mc_eid", "ref", "src"
}

UNWANTED_EXTENSIONS = (".pdf", ".jpg", ".jpeg", ".png", ".gif", ".svg", ".zip", ".rar", ".7z", ".mp4", ".mp3")

def normalize_url(url: str) -> str:
    """
    Normalize a URL by:
      - enforcing scheme+netloc+path normalization
      - removing tracking parameters (utm_*, gclid, fbclid, etc.)
      - removing URL fragments
      - sorting remaining query parameters for stable deduplication
    """
    if not isinstance(url, str) or not url.strip():
        return None

    url = url.strip()

    # Ensure it looks like http(s)
    if not (url.startswith("http://") or url.startswith("https://")):
        return None

    p = urlparse(url)

    # Remove fragment
    fragment = ""

    # Filter and sort query params
    q = []
    for k, v in parse_qsl(p.query, keep_blank_values=True):
        if k.lower() in TRACKING_PARAMS:
            continue
        q.append((k, v))
    q.sort()

    normalized = urlunparse((
        p.scheme.lower(),
        p.netloc.lower(),
        p.path.rstrip("/") or "/",
        p.params,
        urlencode(q, doseq=True),
        fragment
    ))
    return normalized

def extract_domain(url: str) -> str:
    try:
        return urlparse(url).netloc.lower()
    except Exception:
        return None

def is_unwanted_url(url: str) -> bool:
    """Return True if URL should be filtered out (file types, etc.)."""
    if not url:
        return True
    u = url.lower()
    if any(u.endswith(ext) for ext in UNWANTED_EXTENSIONS):
        return True
    return False

def ym_in_text(year_month: str, text: str) -> bool:
    """
    Heuristic: check if the target YYYY-MM token appears in title/snippet.
    This helps enforce the selected month when CSE returns loosely-matched results.
    """
    if not text:
        return False
    return year_month in text

# ----------------------------
# 5-2. Build a Candidate URL DataFrame
# ----------------------------
# Expected input: `raw_results` from discover_candidate_urls(...)
#
# Example:
# raw_results = discover_candidate_urls(RUN_CONFIG, max_results_per_query=5)
#
# This cell converts to a DataFrame and applies cleaning + dedup.

def preprocess_candidate_urls(raw_results: list, run_config: dict) -> pd.DataFrame:
    """
    Preprocess raw CSE results into a cleaned candidate URL DataFrame.
    """
    if not raw_results:
        return pd.DataFrame()

    df = pd.DataFrame(raw_results).copy()

    # Basic sanity checks
    for col in ["url", "title", "snippet", "country", "year_month"]:
        if col not in df.columns:
            df[col] = None

    # Normalize URLs
    df["url_raw"] = df["url"]
    df["url"] = df["url"].apply(normalize_url)

    # Drop invalid / unwanted
    df = df[df["url"].notna()]
    df = df[~df["url"].apply(is_unwanted_url)]

    # Attach domain
    df["domain"] = df["url"].apply(extract_domain)

    # Optional month enforcement (heuristic):
    # Keep rows where YYYY-MM appears in either title or snippet OR in the URL itself.
    # This is conservative and can be relaxed if recall is too low.
    df["month_match"] = df.apply(
        lambda r: (
            ym_in_text(r["year_month"], r.get("title")) or
            ym_in_text(r["year_month"], r.get("snippet")) or
            (r["year_month"] in (r["url"] or ""))
        ),
        axis=1
    )

    # If you prefer higher recall, set this to False or comment out the filter.
    ENFORCE_MONTH_MATCH = False
    if ENFORCE_MONTH_MATCH:
        df = df[df["month_match"] == True]

    # Deduplicate by normalized URL (keep the first occurrence)
    df = df.sort_values(by=["country", "year_month"]).drop_duplicates(subset=["url"], keep="first")

    # Reset index
    df = df.reset_index(drop=True)

    # Minimal output schema
    out_cols = [
        "country", "year_month", "domain",
        "url", "url_raw", "title", "snippet",
        "display_link", "query", "month_match"
    ]
    out_cols = [c for c in out_cols if c in df.columns]
    return df[out_cols]

# Usage:
# candidate_urls_df = preprocess_candidate_urls(raw_results, RUN_CONFIG)
# candidate_urls_df.head()
#
# Next steps:
#   - robots.txt / policy checks (domain-level)
#   - fetch article text for allowed URLs only


In [9]:
# ============================================================
# 013-6 Lightweight robots.txt Check & Content Fetching (Allowed URLs Only)
# ============================================================
# This section performs:
#   (1) A lightweight robots.txt permission check per domain, and
#   (2) Fetches article HTML / text only for URLs that appear crawl-allowed.
#
# IMPORTANT DISCLAIMER
# --------------------
# This notebook is designed for research and analysis using public information.
# Always respect:
#   - robots.txt directives (minimum requirement)
#   - each website's Terms of Use (may impose stricter limitations than robots.txt)
#   - rate limits and server load (polite crawling)
#
# This step intentionally stays conservative:
#   - If robots.txt cannot be fetched, we default to "unknown" and SKIP
#     (you can relax this if you have explicit permission).
#   - Only fetch a minimal subset of pages and cache results locally.
#
# Output:
#   - fetch_df: URL-level table with robots status and extracted text (if fetched)
#
# ------------------------------------------------------------

import time
import random
import requests
import pandas as pd
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from pathlib import Path
import hashlib
import json

# ----------------------------
# 6-1. Polite crawling settings
# ----------------------------
USER_AGENT = "researchOS-bot/0.1 (+contact: you@example.com)"  # update contact if desired
HEADERS = {"User-Agent": USER_AGENT}

# Rate limiting (seconds). Adjust based on your risk tolerance.
SLEEP_MIN = 1.5
SLEEP_MAX = 4.0

# Local cache directory (HTML/text)
CACHE_DIR = Path("./cache_pages")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

def _sleep():
    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX))

def _hash_key(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()[:24]

# ----------------------------
# 6-2. robots.txt checker (domain-level)
# ----------------------------
_robots_cache = {}  # in-memory cache by domain

def can_fetch_url(url: str, user_agent: str = USER_AGENT, timeout: int = 15) -> dict:
    """
    Check whether robots.txt allows fetching the given URL.
    Returns a dict with:
      - robots_ok: True/False/None (None = unknown)
      - robots_url: robots.txt URL
      - robots_error: error message if any
    """
    if not url:
        return {"robots_ok": None, "robots_url": None, "robots_error": "empty_url"}

    p = urlparse(url)
    domain = p.netloc.lower()
    robots_url = f"{p.scheme}://{domain}/robots.txt"

    if domain in _robots_cache:
        rp = _robots_cache[domain]
        if rp is None:
            return {"robots_ok": None, "robots_url": robots_url, "robots_error": "robots_unavailable_cached"}
        return {"robots_ok": rp.can_fetch(user_agent, url), "robots_url": robots_url, "robots_error": None}

    rp = RobotFileParser()
    rp.set_url(robots_url)

    try:
        # RobotFileParser internally fetches robots.txt when read() is called,
        # but it uses urllib and does not set headers. To keep control, we fetch
        # robots.txt ourselves and parse it manually.
        resp = requests.get(robots_url, headers=HEADERS, timeout=timeout)
        if resp.status_code != 200:
            _robots_cache[domain] = None
            return {"robots_ok": None, "robots_url": robots_url, "robots_error": f"robots_http_{resp.status_code}"}

        rp.parse(resp.text.splitlines())
        _robots_cache[domain] = rp
        return {"robots_ok": rp.can_fetch(user_agent, url), "robots_url": robots_url, "robots_error": None}

    except Exception as e:
        _robots_cache[domain] = None
        return {"robots_ok": None, "robots_url": robots_url, "robots_error": str(e)}

# ----------------------------
# 6-3. Minimal HTML fetch + text extraction
# ----------------------------
def fetch_html(url: str, timeout: int = 30) -> str:
    """
    Fetch HTML for a single URL, with basic error handling.
    Returns HTML string or None.
    """
    resp = requests.get(url, headers=HEADERS, timeout=timeout)
    resp.raise_for_status()
    # Basic guard: only accept HTML-like responses
    ctype = (resp.headers.get("Content-Type") or "").lower()
    if "text/html" not in ctype and "application/xhtml" not in ctype:
        return None
    return resp.text

def extract_main_text(html: str) -> str:
    """
    Extract a readable text representation from HTML.
    This is a simple heuristic approach; you may replace with `trafilatura`
    or `readability-lxml` later for better extraction.
    """
    if not html:
        return ""

    soup = BeautifulSoup(html, "lxml")

    # Remove non-content elements
    for tag in soup(["script", "style", "noscript", "header", "footer", "svg"]):
        tag.decompose()

    text = soup.get_text(separator="\n")
    # Normalize whitespace
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    return text

# ----------------------------
# 6-4. On-disk caching
# ----------------------------
def cache_paths(url: str):
    key = _hash_key(url)
    return (
        CACHE_DIR / f"{key}.meta.json",
        CACHE_DIR / f"{key}.html",
        CACHE_DIR / f"{key}.txt",
    )

def load_from_cache(url: str):
    meta_p, html_p, txt_p = cache_paths(url)
    if meta_p.exists() and txt_p.exists():
        meta = json.loads(meta_p.read_text(encoding="utf-8"))
        txt = txt_p.read_text(encoding="utf-8")
        html = html_p.read_text(encoding="utf-8") if html_p.exists() else None
        return meta, html, txt
    return None, None, None

def save_to_cache(url: str, meta: dict, html: str, txt: str):
    meta_p, html_p, txt_p = cache_paths(url)
    meta_p.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
    if html is not None:
        html_p.write_text(html, encoding="utf-8")
    txt_p.write_text(txt or "", encoding="utf-8")

raw_results = discover_candidate_urls(
    RUN_CONFIG,
    max_results_per_query=5
)

print(f"Raw results: {len(raw_results)}")

# ----------------------------
# 6-5. Main runner: robots check + fetch allowed
# ----------------------------
def check_and_fetch(candidate_urls_df: pd.DataFrame,
                    max_pages: int = 200,
                    skip_if_robots_unknown: bool = True) -> pd.DataFrame:
    """
    For each candidate URL:
      - check robots.txt permission
      - fetch + extract text only if allowed
      - cache all fetched results
    """
    if candidate_urls_df is None or candidate_urls_df.empty:
        return pd.DataFrame()

    rows = []
    n = min(len(candidate_urls_df), max_pages)

    for i in range(n):
        r = candidate_urls_df.iloc[i]
        url = r["url"]

        # robots check
        robots_info = can_fetch_url(url)

        robots_ok = robots_info["robots_ok"]
        if robots_ok is None and skip_if_robots_unknown:
            rows.append({**r.to_dict(), **robots_info, "fetched": False, "text": None, "fetch_error": "robots_unknown_skip"})
            continue
        if robots_ok is False:
            rows.append({**r.to_dict(), **robots_info, "fetched": False, "text": None, "fetch_error": "robots_disallow"})
            continue

        # cache lookup
        meta_cached, html_cached, txt_cached = load_from_cache(url)
        if txt_cached is not None:
            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": True,
                "from_cache": True,
                "text": txt_cached,
                "fetch_error": None
            })
            continue

        # fetch
        try:
            _sleep()
            html = fetch_html(url)
            if html is None:
                rows.append({**r.to_dict(), **robots_info, "fetched": False, "from_cache": False, "text": None, "fetch_error": "non_html"})
                continue

            txt = extract_main_text(html)
            meta = {
                "url": url,
                "fetched_at": datetime.utcnow().isoformat() + "Z",
                "user_agent": USER_AGENT,
                "domain": r.get("domain"),
                "country": r.get("country"),
                "year_month": r.get("year_month"),
            }
            save_to_cache(url, meta, html, txt)

            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": True,
                "from_cache": False,
                "text": txt,
                "fetch_error": None
            })

        except Exception as e:
            rows.append({**r.to_dict(), **robots_info, "fetched": False, "from_cache": False, "text": None, "fetch_error": str(e)})

    fetch_df = pd.DataFrame(rows)

    # Keep only a minimal but useful schema
    keep_cols = [
        "country", "year_month", "domain",
        "url", "title", "snippet",
        "robots_ok", "robots_url", "robots_error",
        "fetched", "from_cache", "fetch_error",
        "text"
    ]
    keep_cols = [c for c in keep_cols if c in fetch_df.columns]
    return fetch_df[keep_cols]

# Usage example:
# fetch_df = check_and_fetch(candidate_urls_df, max_pages=200, skip_if_robots_unknown=True)
# fetch_df.head()
#
# Next steps:
#   - OpenAI startup classification (Yes/No)
#   - Structured extraction for Yes cases

candidate_urls_df = preprocess_candidate_urls(
    raw_results,
    RUN_CONFIG
)

print(f"Candidate URLs after preprocessing: {len(candidate_urls_df)}")
candidate_urls_df.head()


from tqdm.auto import tqdm
from datetime import datetime

def check_and_fetch_with_progress(
    candidate_urls_df: pd.DataFrame,
    max_pages: int = 200,
    skip_if_robots_unknown: bool = True,
    verbose: bool = True
) -> pd.DataFrame:
    """
    robots.txt check + content fetch with visible progress.

    Progress indicators:
      - tqdm progress bar (overall)
      - short per-URL status messages (optional)
    """
    if candidate_urls_df is None or candidate_urls_df.empty:
        print("⚠️ No candidate URLs to process.")
        return pd.DataFrame()

    rows = []
    n = min(len(candidate_urls_df), max_pages)

    iterator = tqdm(
        range(n),
        desc="Fetching allowed pages",
        unit="url"
    )

    for i in iterator:
        r = candidate_urls_df.iloc[i]
        url = r["url"]

        iterator.set_postfix_str(url[:60] + ("…" if len(url) > 60 else ""))

        # ----------------------------
        # robots.txt check
        # ----------------------------
        robots_info = can_fetch_url(url)
        robots_ok = robots_info["robots_ok"]

        if robots_ok is None and skip_if_robots_unknown:
            if verbose:
                print(f"⏭️  SKIP (robots unknown): {url}")
            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": False,
                "from_cache": False,
                "text": None,
                "fetch_error": "robots_unknown_skip"
            })
            continue

        if robots_ok is False:
            if verbose:
                print(f"🚫 DISALLOWED by robots.txt: {url}")
            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": False,
                "from_cache": False,
                "text": None,
                "fetch_error": "robots_disallow"
            })
            continue

        # ----------------------------
        # cache lookup
        # ----------------------------
        meta_cached, html_cached, txt_cached = load_from_cache(url)
        if txt_cached is not None:
            if verbose:
                print(f"📦 CACHE HIT: {url}")
            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": True,
                "from_cache": True,
                "text": txt_cached,
                "fetch_error": None
            })
            continue

        # ----------------------------
        # fetch & extract
        # ----------------------------
        try:
            if verbose:
                print(f"🌐 FETCHING: {url}")

            _sleep()
            html = fetch_html(url)
            if html is None:
                if verbose:
                    print(f"⚠️  NON-HTML content skipped: {url}")
                rows.append({
                    **r.to_dict(),
                    **robots_info,
                    "fetched": False,
                    "from_cache": False,
                    "text": None,
                    "fetch_error": "non_html"
                })
                continue

            text = extract_main_text(html)

            meta = {
                "url": url,
                "fetched_at": datetime.utcnow().isoformat() + "Z",
                "user_agent": USER_AGENT,
                "domain": r.get("domain"),
                "country": r.get("country"),
                "year_month": r.get("year_month"),
            }

            save_to_cache(url, meta, html, text)

            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": True,
                "from_cache": False,
                "text": text,
                "fetch_error": None
            })

            if verbose:
                print(f"✅ FETCHED ({len(text)} chars)")

        except Exception as e:
            if verbose:
                print(f"❌ ERROR fetching {url}: {e}")
            rows.append({
                **r.to_dict(),
                **robots_info,
                "fetched": False,
                "from_cache": False,
                "text": None,
                "fetch_error": str(e)
            })

    fetch_df = pd.DataFrame(rows)

    keep_cols = [
        "country", "year_month", "domain",
        "url", "title", "snippet",
        "robots_ok", "robots_url",
        "fetched", "from_cache", "fetch_error",
        "text"
    ]
    keep_cols = [c for c in keep_cols if c in fetch_df.columns]

    print(f"\n✅ Completed: {len(fetch_df)} URLs processed")
    print(fetch_df["fetched"].value_counts(dropna=False))

    return fetch_df[keep_cols]

fetch_df = check_and_fetch_with_progress(
    candidate_urls_df,
    max_pages=100,
    skip_if_robots_unknown=True,
    verbose=True   # False にすると tqdm のみ
)

Raw results: 240
Candidate URLs after preprocessing: 214


Fetching allowed pages:   0%|          | 0/100 [00:00<?, ?url/s]

📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000005.000125075.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000021.000091867.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000267.000010548.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000022.000051834.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000008.000118415.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000021.000044156.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000977.000016451.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000172.000025017.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000397.000000204.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000079.000040785.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000012.000053340.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000152.000048792.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000019.000055047.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000006.0000755

In [12]:
# ============================================================
# 013-7 (Improved) OpenAI Startup Classification with Logs
# ============================================================
import json
import hashlib
from pathlib import Path
from datetime import datetime

import json
from typing import Optional

def call_openai_for_startup_structuring(
    url: str,
    country: Optional[str],
    year_month: Optional[str],
    title: Optional[str],
    text: Optional[str],
    model: str = "gpt-4.1-mini"
) -> dict:
    """
    Call OpenAI to classify whether the content represents a startup,
    and extract structured information if applicable.
    """

    if not text or not isinstance(text, str):
        raise ValueError("Empty or invalid text input")

    system_prompt = """
You are an expert startup analyst.

Your task:
1. Decide whether the content describes a startup company.
2. If YES, extract structured startup information.
3. If NO, clearly explain why.

Definition of a startup:
- A relatively young company or venture
- Typically innovation-driven or growth-oriented
- Often mentions fundraising, venture capital, new products, or early-stage expansion

Output rules:
- Output MUST be valid JSON.
- Do NOT include markdown.
- Do NOT include commentary outside JSON.
- Use null for unknown values.
- Use empty arrays [] where appropriate.
"""

    user_prompt = f"""
URL: {url}
Country: {country}
Year-Month: {year_month}

Title:
{title}

Content:
{text[:12000]}

Instructions:
- First decide if this is a startup.
- If not a startup, set "is_startup" to "No" and explain briefly.
- If a startup, set "is_startup" to "Yes" and fill all applicable fields.
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_prompt.strip()}
        ],
        temperature=0
    )

    raw = response.choices[0].message.content.strip()

    try:
        data = json.loads(raw)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON from OpenAI: {e}\nRaw output:\n{raw}")

    # ---- Minimal schema normalization ----
    result = {
        "is_startup": data.get("is_startup", "No"),
        "confidence": float(data.get("confidence", 0.5)),

        "decision_rationale": data.get("decision_rationale"),

        "company_name": data.get("company_name"),
        "country": country,

        "business_summary": data.get("business_summary"),
        "service_or_product": data.get("service_or_product"),
        "target_market": data.get("target_market"),
        "customer_segments": data.get("customer_segments"),

        "funding_round": data.get("funding_round"),
        "funding_amount": data.get("funding_amount"),
        "funding_amount_currency": data.get("funding_amount_currency"),
        "total_funding_to_date": data.get("total_funding_to_date"),
        "valuation": data.get("valuation"),

        "investors": data.get("investors", []) or [],
        "announcement_date": data.get("announcement_date"),

        "source_urls": [url]
    }

    return result


LLM_CACHE_DIR = Path("./cache_llm")
LLM_CACHE_DIR.mkdir(parents=True, exist_ok=True)

def _llm_cache_key(url: str) -> str:
    return hashlib.sha256(url.encode("utf-8")).hexdigest()[:24]

def _llm_cache_path(url: str) -> Path:
    return LLM_CACHE_DIR / f"{_llm_cache_key(url)}.json"

def load_llm_cache(url: str):
    """
    Load cached LLM result for a URL.
    Returns dict or None.
    """
    if not url:
        return None
    p = _llm_cache_path(url)
    if not p.exists():
        return None
    try:
        return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        return None

def save_llm_cache(url: str, payload: dict):
    """
    Save LLM result payload to cache.
    """
    if not url or payload is None:
        return

    p = _llm_cache_path(url)
    # attach minimal metadata
    obj = dict(payload)
    obj["_cache_saved_at_utc"] = datetime.utcnow().isoformat() + "Z"
    obj["_cache_url"] = url

    p.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")
from tqdm.auto import tqdm
from datetime import datetime

def run_llm_on_fetched_pages_with_logs(
    fetch_df: pd.DataFrame,
    model: str = "gpt-4.1-mini",
    max_pages: int = 200,
    only_fetched: bool = True,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Run OpenAI classification + extraction with visible logs.

    Logs:
      - tqdm progress bar (overall)
      - per-URL status (cache / call / success / error)
    """
    if fetch_df is None or fetch_df.empty:
        print("⚠️ fetch_df is empty.")
        return pd.DataFrame()

    df = fetch_df.copy()
    if only_fetched:
        df = df[df.get("fetched", False) == True].copy()

    if df.empty:
        print("⚠️ No fetched pages to process (fetched==True).")
        return pd.DataFrame()

    df = df.reset_index(drop=True)
    n = min(len(df), max_pages)

    results = []
    iterator = tqdm(range(n), desc="LLM classify & extract", unit="url")

    for i in iterator:
        row = df.iloc[i]
        url = row["url"]
        short_url = url[:60] + ("…" if len(url) > 60 else "")
        iterator.set_postfix_str(short_url)

        # ----------------------------
        # Cache check
        # ----------------------------
        cached = load_llm_cache(url)
        if cached is not None:
            if verbose:
                print(f"📦 CACHE HIT: {short_url}")
            cached["_from_cache"] = True
            results.append(cached)
            continue

        # ----------------------------
        # OpenAI API call
        # ----------------------------
        try:
            if verbose:
                print(f"🤖 CALL OpenAI: {short_url}")

            payload = call_openai_for_startup_structuring(
                url=url,
                country=row.get("country"),
                year_month=row.get("year_month"),
                title=row.get("title"),
                text=row.get("text"),
                model=model
            )

            payload["_from_cache"] = False
            save_llm_cache(url, payload)

            if verbose:
                print(f"✅ SUCCESS | is_startup={payload.get('is_startup')} "
                      f"| confidence={payload.get('confidence')}")

            results.append(payload)

        except Exception as e:
            if verbose:
                print(f"❌ ERROR OpenAI: {short_url} | {e}")

            results.append({
                "is_startup": "No",
                "confidence": 0.0,
                "decision_rationale": f"LLM_ERROR: {e}",
                "company_name": None,
                "country": row.get("country"),
                "business_summary": None,
                "service_or_product": None,
                "target_market": None,
                "customer_segments": None,
                "funding_round": None,
                "funding_amount": None,
                "funding_amount_currency": None,
                "total_funding_to_date": None,
                "valuation": None,
                "investors": [],
                "announcement_date": None,
                "source_urls": [url],
                "_from_cache": False
            })

    llm_results_df = pd.DataFrame(results)

    # ----------------------------
    # Summary
    # ----------------------------
    print("\n📊 Startup classification summary:")
    display(llm_results_df["is_startup"].value_counts(dropna=False))

    print("\n📦 Cache usage:")
    display(llm_results_df["_from_cache"].value_counts(dropna=False))

    return llm_results_df

llm_results_df = run_llm_on_fetched_pages_with_logs(
    fetch_df,
    model="gpt-4.1-mini",
    max_pages=100,
    verbose=True
)

LLM classify & extract:   0%|          | 0/100 [00:00<?, ?url/s]

📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000005.000125075.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000021.000091867.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000267.000010548.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000022.000051834.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000008.000118415.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000021.000044156.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000977.000016451.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000172.000025017.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000397.000000204.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000079.000040785.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000012.000053340.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000152.000048792.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000019.000055047.html
📦 CACHE HIT: https://prtimes.jp/main/html/rd/p/000000006.0000755

is_startup
Yes    80
No     20
Name: count, dtype: int64


📦 Cache usage:


_from_cache
True     71
False    29
Name: count, dtype: int64

In [13]:
startup_df = llm_results_df[llm_results_df["is_startup"] == "Yes"].copy()

display(
    startup_df[
        ["company_name", "business_summary", "funding_round",
         "funding_amount", "confidence"]
    ].sort_values("confidence", ascending=False).head(20)
)


,company_name,business_summary,funding_round,funding_amount,confidence
0,ペリオセラピア株式会社,A drug discovery venture originating from Osak...,Additional funding round,約1億円,0.95
1,イノバセル株式会社,Innovacell is a regenerative medicine venture ...,Series D,10.6億円,0.95
3,bitBiome株式会社,bitBiome is a biotechnology company leveraging...,Global Seed Extension Second Close,"400,000,000",0.95
4,株式会社Power Diamond Systems,Power Diamond Systems is a startup conducting ...,Third-party allotment (2nd close),100000000,0.95
7,株式会社Legalscape,株式会社Legalscape provides an AI-powered legal re...,Secondary transaction,"160,000,000",0.95
8,bythen,bythen is an AI technology startup specializin...,None,None,0.95
10,株式会社すきだよ,株式会社すきだよ is a startup focused on solving socia...,"Not explicitly stated, but funding was raised ...",None,0.95
20,FirstShift,FirstShift operates an AI matching platform 'A...,None,"50,000,000",0.95
18,Manabie International Pte. Ltd.,Manabie is an EdTech startup providing an educ...,Series B,33億円,0.95
19,Medteria,Medteria provides a healthcare communication c...,Series Unknown (first disclosed round),"100,000,000",0.95


In [14]:
no_df = llm_results_df[llm_results_df["is_startup"] == "No"].copy()

display(
    no_df[
        ["decision_rationale", "source_urls"]
    ].head(10)
)


,decision_rationale,source_urls
2,"The company, 株式会社ユーザベース (Uzabase, Inc.), was e...",[https://prtimes.jp/main/html/rd/p/000000267.0...
5,"The company, 株式会社インベストメントブリッジ, was established...",[https://prtimes.jp/main/html/rd/p/000000021.0...
6,"The company, パーソルホールディングス株式会社, is a long-estab...",[https://prtimes.jp/main/html/rd/p/000000977.0...
11,The source describes the 'スタートアップ都市推進協議会' (Sta...,[https://prtimes.jp/main/html/rd/p/000000152.0...
15,株式会社ゼロワンブースター (01Booster) was established in 2...,[https://prtimes.jp/main/html/rd/p/000000709.0...
23,Creww was founded in 2012 and has established ...,[https://thebridge.jp/en/2025/01/creww-launche...
25,"X&KSK is a venture capital firm, not a startup...",[https://thebridge.jp/en/2025/01/xksk-led-by-k...
30,The source text describes BRIDGE as a media an...,[https://thebridge.jp/2025/01/happy-new-year-2...
32,The source text discusses internal innovation ...,[https://thebridge.jp/2025/03/01bcf2024-newbiz...
46,株式会社ケップルは2015年設立で資本金4億344万円の情報通信企業であり、スタートアップ動...,[https://prtimes.jp/main/html/rd/p/000000089.0...


In [15]:
startup_df["funding_round"].value_counts(dropna=False).head(10)


funding_round
None                                                                                                                   48
Series B                                                                                                                4
Series C                                                                                                                4
Pre-Series A                                                                                                            3
Series B Extension                                                                                                      2
Additional funding round                                                                                                1
Global Seed Extension Second Close                                                                                      1
Series D                                                                                                                1
J-KISS con

In [16]:
# ============================================================
# 013-7.5 OpenAI API: Translation to English (with Cache)
# ============================================================
# This section translates Japanese (or non-English) text fields into English
# using the OpenAI API.
#
# Design principles:
#   - Preserve original text (no overwrite)
#   - Translate only when non-English is detected
#   - Cache translations to avoid repeated API costs
#   - Use English as the primary display language downstream
#
# Input:
#   - llm_results_df
#
# Output:
#   - llm_results_df (augmented with *_en fields)
#
# ------------------------------------------------------------

import hashlib
from pathlib import Path
from tqdm.auto import tqdm

TRANSLATION_CACHE_DIR = Path("./cache_translation")
TRANSLATION_CACHE_DIR.mkdir(parents=True, exist_ok=True)

def _translation_cache_path(text: str) -> Path:
    key = hashlib.sha256(text.encode("utf-8")).hexdigest()[:24]
    return TRANSLATION_CACHE_DIR / f"{key}.txt"

def load_translation_cache(text: str):
    p = _translation_cache_path(text)
    if p.exists():
        return p.read_text(encoding="utf-8")
    return None

def save_translation_cache(text: str, translated: str):
    p = _translation_cache_path(text)
    p.write_text(translated, encoding="utf-8")

# Simple non-English heuristic (CJK)
_CJK_RE = re.compile(r"[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]")

def needs_translation(text: str) -> bool:
    if not text or not isinstance(text, str):
        return False
    return bool(_CJK_RE.search(text))

def translate_to_english(text: str, model: str = "gpt-4.1-mini") -> str:
    cached = load_translation_cache(text)
    if cached is not None:
        return cached

    prompt = f"""
Translate the following text into clear, professional English.
Preserve the original meaning faithfully.
Do NOT add interpretation or extra information.

TEXT:
{text}
"""

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": prompt.strip()}
        ],
        temperature=0
    )

    translated = resp.choices[0].message.content.strip()
    save_translation_cache(text, translated)
    return translated

# ----------------------------
# Run translation on selected fields
# ----------------------------
FIELDS_TO_TRANSLATE = [
    "business_summary",
    "service_or_product",
    "target_market",
    "customer_segments"
]

df = llm_results_df.copy()

for field in FIELDS_TO_TRANSLATE:
    en_field = f"{field}_en"
    translated = []

    iterator = tqdm(df[field].fillna("").tolist(), desc=f"Translating {field}", unit="row")
    for text in iterator:
        if needs_translation(text):
            translated.append(translate_to_english(text))
        else:
            translated.append(text or None)

    df[en_field] = translated

llm_results_df = df

print("✅ Translation completed and cached.")


Translating business_summary:   0%|          | 0/100 [00:00<?, ?row/s]

Translating service_or_product:   0%|          | 0/100 [00:00<?, ?row/s]

Translating target_market:   0%|          | 0/100 [00:00<?, ?row/s]

Translating customer_segments:   0%|          | 0/100 [00:00<?, ?row/s]

✅ Translation completed and cached.


In [17]:
# ============================================================
# 013-7.6 OpenAI API: Company Name Translation (JP/CJK -> EN) with Cache
# ============================================================

import re
import hashlib
from pathlib import Path
from tqdm.auto import tqdm

# --- language detection (self-contained to avoid NameError) ---
_CJK_RE = re.compile(r"[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]")

def contains_cjk(text: str) -> bool:
    return bool(text) and bool(_CJK_RE.search(str(text)))

COMPANY_NAME_TRANSLATION_CACHE = Path("./cache_company_name_translation")
COMPANY_NAME_TRANSLATION_CACHE.mkdir(parents=True, exist_ok=True)

def _company_name_cache_path(name: str) -> Path:
    key = hashlib.sha256(name.encode("utf-8")).hexdigest()[:24]
    return COMPANY_NAME_TRANSLATION_CACHE / f"{key}.txt"

def load_company_name_cache(name: str):
    p = _company_name_cache_path(name)
    return p.read_text(encoding="utf-8") if p.exists() else None

def save_company_name_cache(name: str, translated: str):
    p = _company_name_cache_path(name)
    p.write_text(translated, encoding="utf-8")

def needs_company_name_translation(name: str) -> bool:
    if not name or not isinstance(name, str):
        return False
    # If any CJK exists, we want an English display version.
    return contains_cjk(name)

def _translate_once(name: str, model: str) -> str:
    prompt = f"""
Translate the following company name into natural, concise English.

Rules:
- Do NOT add legal suffixes such as "Inc.", "Ltd.", etc., unless clearly implied.
- If the name is a coined brand name, transliterate appropriately.
- Return ONLY the English name (ASCII letters/numbers if possible).

Company name:
{name}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a professional business translator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    return resp.choices[0].message.content.strip()

def translate_company_name_to_english(name: str, model: str = "gpt-4.1-mini") -> str:
    cached = load_company_name_cache(name)
    if cached is not None:
        return cached

    translated = _translate_once(name, model=model)

    # If the output still contains CJK, retry once with a stricter instruction.
    if contains_cjk(translated):
        prompt_retry = f"""
Convert the following Japanese company name into an English rendering.

Rules:
- Output MUST be in English (no Japanese characters).
- If you cannot find an official English name, provide a romanized/transliterated version.
- Return ONLY the name.

Company name:
{name}
""".strip()

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a professional business translator."},
                {"role": "user", "content": prompt_retry}
            ],
            temperature=0
        )
        translated = resp.choices[0].message.content.strip()

    save_company_name_cache(name, translated)
    return translated

# ----------------------------
# Run company name translation
# ----------------------------
df = llm_results_df.copy()

translated_names = []
iterator = tqdm(df["company_name"].fillna("").tolist(), desc="Translating company names", unit="name")

for name in iterator:
    name = (name or "").strip()
    if not name:
        translated_names.append(None)
        continue

    if needs_company_name_translation(name):
        translated_names.append(translate_company_name_to_english(name))
    else:
        # Already non-CJK (likely English), keep as-is
        translated_names.append(name)

df["company_name_english_llm"] = translated_names
llm_results_df = df

print("✅ Company name translation completed and cached.")


Translating company names:   0%|          | 0/100 [00:00<?, ?name/s]

✅ Company name translation completed and cached.


In [18]:
# ============================================================
# 013-8 Entity Resolution, Formatting, and Deduplication
# ============================================================
# This section consolidates URL-level startup records into a clean company-level dataset by:
#   - Normalizing company names (deterministic)
#   - Resolving entities (name-based)
#   - Deduplicating and aggregating multiple sources per company
#   - Parsing funding amounts into numeric values
#   - Translating company names and key descriptive fields into English (OpenAI API)
#   - Providing display-friendly formatting (e.g., 1,234,567)
#
# Input:
#   - llm_results_df (output of Section 013-7)
#
# Output:
#   - company_level_df (one row per company_key)
#
# Notes:
#   - Original text is always preserved.
#   - English translations are stored in separate *_en or *_english_llm fields.
#   - Display columns are English-first; `company_name_display` is ENGLISH-ONLY
#     (no Japanese/CJK fallback).
#
# ------------------------------------------------------------

import re
import unicodedata
import hashlib
from pathlib import Path
from hashlib import sha1

import pandas as pd
from tqdm.auto import tqdm

# ----------------------------
# 8-1. Regex helpers (language detection)
# ----------------------------
_CJK_RE = re.compile(r"[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]")   # JP + CJK
_LATIN_RE = re.compile(r"[A-Za-z]")

def contains_cjk(text: str) -> bool:
    return bool(text) and bool(_CJK_RE.search(str(text)))

def contains_latin(text: str) -> bool:
    return bool(text) and bool(_LATIN_RE.search(str(text)))

# ----------------------------
# 8-2. Company name normalization (deterministic)
# ----------------------------
LEGAL_SUFFIX_PATTERNS = [
    r"\binc\b\.?", r"\bincorporated\b",
    r"\bltd\b\.?", r"\blimited\b",
    r"\bco\b\.?", r"\bcompany\b",
    r"\bcorp\b\.?", r"\bcorporation\b",
    r"\bllc\b", r"\bplc\b", r"\bgmbh\b",
    r"\bs\.a\.?\b", r"\bs\.a\.s\.?\b",
    r"\bkk\b", r"\bkaisha\b",
    r"株式会社", r"（株）", r"\(株\)", r"有限会社", r"合同会社"
]

def split_company_name_bilingual(name: str):
    """
    Split a company name into:
      - company_name_local: if it contains Japanese/CJK characters
      - company_name_english: if it contains Latin characters
    If the input contains both, both fields may be populated.
    """
    if not name:
        return None, None
    s = unicodedata.normalize("NFKC", str(name)).strip()
    return (s if contains_cjk(s) else None), (s if contains_latin(s) else None)

def normalize_company_name(name: str) -> str:
    """
    Normalize a company name into a canonical string for deterministic deduplication.
    """
    if not name:
        return None

    s = unicodedata.normalize("NFKC", str(name)).strip()

    # Remove legal suffixes
    for pat in LEGAL_SUFFIX_PATTERNS:
        s = re.sub(pat, "", s, flags=re.IGNORECASE)

    # Keep alphanumerics, CJK characters, and spaces only
    s = re.sub(r"[^\w\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()

    return s or None

def make_company_key(country: str, company_name_norm: str) -> str:
    """
    Create a stable, deterministic company key using country + normalized name.
    """
    if not company_name_norm:
        return None
    base = f"{(country or '').upper()}::{company_name_norm}"
    return sha1(base.encode("utf-8")).hexdigest()[:16]

# ----------------------------
# 8-3. Funding amount parsing (numeric) + display formatting
# ----------------------------
MULTIPLIERS = {
    "k": 1e3, "m": 1e6, "b": 1e9,
    "million": 1e6, "billion": 1e9,
    "億": 1e8, "万": 1e4
}

def parse_funding_amount_to_number(text: str):
    """
    Best-effort parsing of funding amount text into an integer number.
    Returns None if parsing fails.
    Examples:
      - "10.6億円" -> 1,060,000,000 (stored as 1060000000)
      - "4億円"    -> 400,000,000
      - "USD 12.5M"-> 12,500,000
      - "¥500M"    -> 500,000,000
    """
    if not text or not isinstance(text, str):
        return None

    t = text.strip().replace(",", "")
    t_low = t.lower()

    # Japanese formats: "10.6億円", "3000万円"
    m = re.search(r"([\d\.]+)\s*(億|万)\s*円", t_low)
    if m:
        value = float(m.group(1)) * MULTIPLIERS[m.group(2)]
        return int(round(value))

    # Western formats: "12.5M", "1.2B", "500k"
    m = re.search(r"([\d\.]+)\s*(k|m|b|million|billion)\b", t_low)
    if m:
        value = float(m.group(1)) * MULTIPLIERS[m.group(2)]
        return int(round(value))

    # Fallback: extract a large integer token
    m = re.search(r"\b(\d{6,})\b", t_low)
    if m:
        try:
            return int(m.group(1))
        except Exception:
            return None

    return None

def format_int_commas(x):
    """
    Format integers with thousand separators: 1234567 -> "1,234,567"
    """
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    try:
        return f"{int(x):,}"
    except Exception:
        return None

# ----------------------------
# 8-4. OpenAI translation layer with caching
# ----------------------------
# Requirements:
#   - `client` must be defined (from Section 013-4 OpenAI setup).
#   - Translation is best-effort and NOT guaranteed to match official naming.
#   - We cache translations to avoid repeated API cost.
#
# Translation policy in this cell:
#   - Company names: if ANY CJK is present, ALWAYS produce an English display name.
#   - Descriptive fields: translate when CJK is present and Latin is not present.

TRANSLATION_CACHE_DIR = Path("./cache_translation")
TRANSLATION_CACHE_DIR.mkdir(parents=True, exist_ok=True)

COMPANY_NAME_TRANSLATION_CACHE_DIR = Path("./cache_company_name_translation")
COMPANY_NAME_TRANSLATION_CACHE_DIR.mkdir(parents=True, exist_ok=True)

def _cache_path_for_text(cache_dir: Path, text: str) -> Path:
    key = hashlib.sha256(text.encode("utf-8")).hexdigest()[:24]
    return cache_dir / f"{key}.txt"

def translate_text_to_english(text: str, model: str = "gpt-4.1-mini") -> str:
    """
    Translate arbitrary text into English. Uses cache.
    """
    if not text or not isinstance(text, str):
        return None

    p = _cache_path_for_text(TRANSLATION_CACHE_DIR, text)
    if p.exists():
        return p.read_text(encoding="utf-8")

    prompt = f"""
Translate the following text into clear, professional English.
Preserve the original meaning faithfully.
Do NOT add interpretation or extra information.
Return ONLY the translated text.

TEXT:
{text}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    translated = resp.choices[0].message.content.strip()
    p.write_text(translated, encoding="utf-8")
    return translated

def translate_company_name_to_english(name: str, model: str = "gpt-4.1-mini") -> str:
    """
    Translate a Japanese/CJK company name into natural English. Uses cache.
    Retries once if the output still contains CJK characters.
    """
    if not name or not isinstance(name, str):
        return None

    p = _cache_path_for_text(COMPANY_NAME_TRANSLATION_CACHE_DIR, name)
    if p.exists():
        return p.read_text(encoding="utf-8")

    prompt = f"""
Translate the following company name into natural, concise English.

Rules:
- Do NOT add legal suffixes such as "Inc.", "Ltd.", etc., unless clearly implied.
- If the name is a coined brand name, transliterate appropriately.
- Return ONLY the English name (no Japanese characters).

Company name:
{name}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a professional business translator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    translated = resp.choices[0].message.content.strip()

    # Retry once if CJK remains
    if contains_cjk(translated):
        prompt_retry = f"""
Convert the following company name into an English rendering.

Rules:
- Output MUST be in English (no Japanese characters).
- If you cannot find an official English name, provide a romanized/transliterated version.
- Return ONLY the name.

Company name:
{name}
""".strip()

        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a professional business translator."},
                {"role": "user", "content": prompt_retry}
            ],
            temperature=0
        )
        translated = resp.choices[0].message.content.strip()

    p.write_text(translated, encoding="utf-8")
    return translated

def needs_translation_to_english(text: str) -> bool:
    """
    Translate descriptive fields when the text contains CJK and does not contain Latin characters.
    """
    if not text or not isinstance(text, str):
        return False
    return contains_cjk(text) and not contains_latin(text)

# ----------------------------
# 8-5. Prepare URL-level working table (startups only)
# ----------------------------
df = llm_results_df.copy()
df = df[df["is_startup"] == "Yes"].copy()

expected_cols = [
    "company_name", "country", "confidence",
    "business_summary", "service_or_product", "target_market", "customer_segments",
    "funding_round", "funding_amount", "funding_amount_currency",
    "total_funding_to_date", "valuation",
    "investors", "source_urls"
]
for col in expected_cols:
    if col not in df.columns:
        df[col] = None

# Preserve original company name (avoid turning NaN into string "nan")
df["company_name_original"] = df["company_name"].where(df["company_name"].notna(), None)

# Split bilingual forms if present (from the original string)
df[["company_name_local", "company_name_english"]] = df["company_name"].apply(
    lambda x: pd.Series(split_company_name_bilingual(x))
)

# ---- Company name translation (ALWAYS if any CJK exists) ----
company_name_english_llm = []
for name in tqdm(df["company_name"].fillna("").tolist(), desc="Translating company names", unit="name"):
    name = (name or "").strip()
    if not name:
        company_name_english_llm.append(None)
        continue

    if contains_cjk(name):
        company_name_english_llm.append(translate_company_name_to_english(name))
    else:
        # already non-CJK (likely English) -> keep
        company_name_english_llm.append(name)

df["company_name_english_llm"] = company_name_english_llm

# ---- Translate key narrative fields into English (store separately) ----
FIELDS_TO_TRANSLATE = ["business_summary", "service_or_product", "target_market", "customer_segments"]
for field in FIELDS_TO_TRANSLATE:
    en_field = f"{field}_en"
    out = []
    for text in tqdm(df[field].fillna("").tolist(), desc=f"Translating {field}", unit="row"):
        text = (text or "").strip()
        if not text:
            out.append(None)
        elif needs_translation_to_english(text):
            out.append(translate_text_to_english(text))
        else:
            # If already English/mixed, keep original as "display" later; no translation needed
            out.append(None)
    df[en_field] = out

# Deterministic name normalization + company key
df["company_name_norm"] = df["company_name"].apply(normalize_company_name)
df["company_key"] = df.apply(
    lambda r: make_company_key(r.get("country"), r.get("company_name_norm")),
    axis=1
)

# Funding numeric + formatted display
df["funding_amount_numeric"] = df["funding_amount"].apply(parse_funding_amount_to_number)
df["funding_amount_fmt"] = df["funding_amount_numeric"].apply(format_int_commas)

df = df[df["company_key"].notna()].reset_index(drop=True)

print("URL-level startup rows:", len(df))
print("Unique companies:", df["company_key"].nunique())

# ----------------------------
# 8-6. Aggregation helpers
# ----------------------------
def merge_unique_lists(series):
    seen, out = set(), []
    for x in series.dropna():
        items = x if isinstance(x, list) else [x]
        for item in items:
            if item is None:
                continue
            item = str(item).strip()
            if not item or item in seen:
                continue
            seen.add(item)
            out.append(item)
    return out

def pick_highest_confidence_row(g: pd.DataFrame) -> pd.Series:
    return g.sort_values("confidence", ascending=False).iloc[0]

def english_first(en_text: str, original_text: str):
    """
    Prefer English translation when available; otherwise fall back to original.
    """
    return en_text if (en_text is not None and str(en_text).strip()) else (original_text if original_text else None)

# ----------------------------
# 8-7. Company-level aggregation
# ----------------------------
company_rows = []

for company_key, g in df.groupby("company_key", sort=False):
    best = pick_highest_confidence_row(g)

    # ENGLISH-ONLY display name (no CJK fallback)
    company_name_display = (
        best.get("company_name_english_llm")
        or best.get("company_name_english")
        or None
    )

    company_rows.append({
        "company_key": company_key,
        "country": best.get("country"),

        # Company names (English display only; preserve original/local separately)
        "company_name_display": company_name_display,
        "company_name_english_llm": best.get("company_name_english_llm"),
        "company_name_english": best.get("company_name_english"),
        "company_name_local": best.get("company_name_local"),
        "company_name_original": best.get("company_name_original"),
        "company_name_norm": best.get("company_name_norm"),

        # Core narrative (English-first display; preserve original + translated)
        "business_summary": best.get("business_summary"),
        "business_summary_en": best.get("business_summary_en"),
        "business_summary_display": english_first(best.get("business_summary_en"), best.get("business_summary")),

        "service_or_product": best.get("service_or_product"),
        "service_or_product_en": best.get("service_or_product_en"),
        "service_or_product_display": english_first(best.get("service_or_product_en"), best.get("service_or_product")),

        "target_market": best.get("target_market"),
        "target_market_en": best.get("target_market_en"),
        "target_market_display": english_first(best.get("target_market_en"), best.get("target_market")),

        "customer_segments": best.get("customer_segments"),
        "customer_segments_en": best.get("customer_segments_en"),
        "customer_segments_display": english_first(best.get("customer_segments_en"), best.get("customer_segments")),

        # Funding (numeric + formatted)
        "funding_round": best.get("funding_round"),
        "funding_amount_numeric": best.get("funding_amount_numeric"),
        "funding_amount_fmt": format_int_commas(best.get("funding_amount_numeric")),
        "funding_amount_raw": best.get("funding_amount"),
        "funding_amount_currency": best.get("funding_amount_currency"),
        "total_funding_to_date": best.get("total_funding_to_date"),
        "valuation": best.get("valuation"),

        # Aggregated fields
        "investors": merge_unique_lists(g.get("investors", pd.Series([]))),
        "source_urls": merge_unique_lists(g.get("source_urls", pd.Series([]))),

        # Metadata
        "max_confidence": float(g["confidence"].max()),
        "num_sources": int(len(g))
    })

company_level_df = pd.DataFrame(company_rows).sort_values(
    by=["max_confidence", "num_sources"],
    ascending=[False, False]
).reset_index(drop=True)

# Remove rows where we still do not have an English display name
company_level_df = company_level_df[company_level_df["company_name_display"].notna()].copy().reset_index(drop=True)

print("\n✅ Company-level dataset created")
print("Companies:", len(company_level_df))

# Compact overview
display_cols = [
    "company_name_display",
    "funding_round",
    "funding_amount_fmt",
    "funding_amount_currency",
    "max_confidence",
    "num_sources"
]
display(company_level_df[display_cols].head(30))

# ----------------------------
# 8-8. Sanity checks
# ----------------------------
assert company_level_df["company_key"].is_unique, "Duplicate company_key detected."

# Ensure company_name_display contains no CJK characters
bad = company_level_df["company_name_display"].dropna().apply(contains_cjk)
print("Rows with CJK in company_name_display:", int(bad.sum()))
if bad.any():
    display(company_level_df.loc[bad, ["company_name_display", "company_name_local", "company_name_original"]].head(20))

print("Missing funding_amount_numeric:", company_level_df["funding_amount_numeric"].isna().sum())

# Next:
#   - Section 013-9 Output generation (CSV / Parquet / SQLite)


Translating company names:   0%|          | 0/80 [00:00<?, ?name/s]

Translating business_summary:   0%|          | 0/80 [00:00<?, ?row/s]

Translating service_or_product:   0%|          | 0/80 [00:00<?, ?row/s]

Translating target_market:   0%|          | 0/80 [00:00<?, ?row/s]

Translating customer_segments:   0%|          | 0/80 [00:00<?, ?row/s]

URL-level startup rows: 52
Unique companies: 52

✅ Company-level dataset created
Companies: 52


,company_name_display,funding_round,funding_amount_fmt,funding_amount_currency,max_confidence,num_sources
0,Perio Therapia,Additional funding round,"100,000,000",JPY,0.95,1
1,Inovacell,Series D,"1,060,000,000",JPY,0.95,1
2,bitBiome,Global Seed Extension Second Close,"400,000,000",JPY,0.95,1
3,Power Diamond Systems,Third-party allotment (2nd close),"100,000,000",JPY,0.95,1
4,Legalscape,Secondary transaction,"160,000,000",JPY,0.95,1
5,bythen,None,None,None,0.95,1
6,Sukidayo,"Not explicitly stated, but funding was raised ...",None,None,0.95,1
7,Legal Agent,J-KISS convertible note,"50,000,000",JPY,0.95,1
8,Manabie International Pte. Ltd.,Series B,"3,300,000,000",JPY,0.95,1
9,Medteria,Series Unknown (first disclosed round),"100,000,000",JPY,0.95,1


Rows with CJK in company_name_display: 0
Missing funding_amount_numeric: 23


In [19]:
# ============================================================
# 013-9 Output Data Generation (Timestamped Exports)
# ============================================================
# This section exports the cleaned, company-level dataset (and optional URL-level tables)
# into a local `data/` folder with timestamped filenames for reproducibility.
#
# Inputs (expected):
#   - company_level_df  (from Section 013-8)
#   - (optional) df     (URL-level startup table built in Section 013-8)
#   - (optional) fetch_df / llm_results_df for deeper debugging
#
# Outputs:
#   - data/company_level_<YYYYMMDD_HHMMSS>.csv
#   - data/company_level_<YYYYMMDD_HHMMSS>.parquet
#   - (optional) data/url_level_startups_<YYYYMMDD_HHMMSS>.csv
#   - (optional) data/run_manifest_<YYYYMMDD_HHMMSS>.json
#
# Notes:
#   - CSV is convenient for inspection and sharing.
#   - Parquet is efficient for analysis and storage.
#   - A manifest file captures key run metadata for auditability.
#
# ------------------------------------------------------------

from pathlib import Path
from datetime import datetime
import json
import pandas as pd

# ----------------------------
# 9-1. Create output folder + timestamp
# ----------------------------
DATA_DIR = Path("./data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Use UTC timestamp for reproducibility (change to local time if you prefer)
ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")

print(f"📦 Export timestamp (UTC): {ts}")
print(f"📁 Output folder: {DATA_DIR.resolve()}")

# ----------------------------
# 9-2. Basic schema checks
# ----------------------------
if "company_level_df" not in globals() or company_level_df is None or company_level_df.empty:
    raise ValueError("company_level_df is missing or empty. Run Section 013-8 first.")

# Ensure stable column order (optional, but helpful)
PREFERRED_COL_ORDER = [
    "company_key", "country",
    "company_name_display",
    "company_name_english_llm", "company_name_english",
    "company_name_local", "company_name_original",
    "business_summary_display", "service_or_product_display",
    "target_market_display", "customer_segments_display",
    "funding_round",
    "funding_amount_numeric", "funding_amount_fmt", "funding_amount_currency",
    "total_funding_to_date", "valuation",
    "investors", "source_urls",
    "max_confidence", "num_sources"
]
cols = [c for c in PREFERRED_COL_ORDER if c in company_level_df.columns] + \
       [c for c in company_level_df.columns if c not in PREFERRED_COL_ORDER]

export_company_df = company_level_df[cols].copy()

# ----------------------------
# 9-3. File paths (timestamped)
# ----------------------------
company_csv_path = DATA_DIR / f"company_level_{ts}.csv"
company_parquet_path = DATA_DIR / f"company_level_{ts}.parquet"

# Optional: URL-level startup table (from Section 013-8)
url_csv_path = DATA_DIR / f"url_level_startups_{ts}.csv"

# Optional: manifest (run metadata)
manifest_path = DATA_DIR / f"run_manifest_{ts}.json"

# ----------------------------
# 9-4. Export: CSV (pandas 2.x compatible)
# ----------------------------
# Convert list/dict columns to JSON strings for CSV compatibility

def _to_json_str_if_needed(series: pd.Series) -> pd.Series:
    if series.dtype == "object":
        return series.map(
            lambda x: json.dumps(x, ensure_ascii=False)
            if isinstance(x, (list, dict)) else x
        )
    return series

export_company_csv = export_company_df.copy()
for col in export_company_csv.columns:
    export_company_csv[col] = _to_json_str_if_needed(export_company_csv[col])

export_company_csv.to_csv(
    company_csv_path,
    index=False,
    encoding="utf-8-sig"
)
print(f"✅ Saved CSV: {company_csv_path}")


# ----------------------------
# 9-5. Export: Parquet (optional)
# ----------------------------
# Parquet export is optional and depends on the runtime environment.
# If pyarrow / fastparquet is not installed, we gracefully skip.

try:
    export_company_df.to_parquet(company_parquet_path, index=False)
    print(f"✅ Saved Parquet: {company_parquet_path}")
except Exception as e:
    print("ℹ️ Parquet export skipped (optional dependency not installed).")
    print("   Reason:", str(e).split("\n")[0])
    print("   → CSV export is still available and sufficient for most use cases.")

# ----------------------------
# 9-6. Optional export: URL-level startup rows used in 013-8
# ----------------------------
# If you kept `df` in Section 013-8, export it for debugging / traceability.
if "df" in globals() and isinstance(df, pd.DataFrame) and not df.empty:
    url_export = df.copy()

    for col in url_export.columns:
        if url_export[col].dtype == "object":
            url_export[col] = url_export[col].map(
                lambda x: json.dumps(x, ensure_ascii=False)
                if isinstance(x, (list, dict)) else x
            )

    url_export.to_csv(url_csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ Saved URL-level CSV: {url_csv_path}")
else:
    print("ℹ️ Skipped URL-level export (df not found or empty).")


# ----------------------------
# 9-7. Manifest (recommended for auditability)
# ----------------------------
manifest = {
    "timestamp_utc": ts,
    "outputs": {
        "company_csv": str(company_csv_path),
        "company_parquet": str(company_parquet_path),
        "url_level_csv": str(url_csv_path) if url_csv_path.exists() else None
    },
    "row_counts": {
        "company_level_df": int(len(company_level_df)),
        "url_level_df": int(len(df)) if ("df" in globals() and isinstance(df, pd.DataFrame)) else None
    },
    "columns": {
        "company_level_df": list(company_level_df.columns)
    }
}

manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"✅ Saved manifest: {manifest_path}")

# ----------------------------
# 9-8. Quick preview
# ----------------------------
print("\n📌 Preview (top rows):")
display(export_company_df.head(10))


📦 Export timestamp (UTC): 20251229_052907
📁 Output folder: /Users/yuetoya/Desktop/researchOS100-private/notebooks/data
✅ Saved CSV: data/company_level_20251229_052907.csv
ℹ️ Parquet export skipped (optional dependency not installed).
   Reason: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
   → CSV export is still available and sufficient for most use cases.
✅ Saved URL-level CSV: data/url_level_startups_20251229_052907.csv
✅ Saved manifest: data/run_manifest_20251229_052907.json

📌 Preview (top rows):


,company_key,country,company_name_display,company_name_english_llm,company_name_english,company_name_local,company_name_original,business_summary_display,service_or_product_display,target_market_display,...,company_name_norm,business_summary,business_summary_en,service_or_product,service_or_product_en,target_market,target_market_en,customer_segments,customer_segments_en,funding_amount_raw
0,8632be66cb22e714,JP,Perio Therapia,Perio Therapia,None,ペリオセラピア株式会社,ペリオセラピア株式会社,A drug discovery venture originating from Osak...,Antibody drugs targeting pathological periosti...,Triple-negative breast cancer and metastatic r...,...,ペリオセラピア,A drug discovery venture originating from Osak...,None,Antibody drugs targeting pathological periosti...,None,Triple-negative breast cancer and metastatic r...,None,Patients with difficult-to-treat diseases such...,None,約1億円
1,80b9c03f7ffa10ad,JP,Inovacell,Inovacell,None,イノバセル株式会社,イノバセル株式会社,Innovacell is a regenerative medicine venture ...,"ICEF15, a cell therapy product using patient's...",Patients suffering from urge fecal incontinenc...,...,イノバセル,Innovacell is a regenerative medicine venture ...,None,"ICEF15, a cell therapy product using patient's...",None,Patients suffering from urge fecal incontinenc...,None,Healthcare providers and patients requiring tr...,None,10.6億円
2,542fdb9efc59f6e1,JP,bitBiome,bitBiome,bitBiome株式会社,bitBiome株式会社,bitBiome株式会社,bitBiome is a biotechnology company leveraging...,"Microbial genome database (bit-GEM), single-ce...","Biomanufacturing industry, pharmaceutical manu...",...,bitbiome,bitBiome is a biotechnology company leveraging...,None,"Microbial genome database (bit-GEM), single-ce...",None,"Biomanufacturing industry, pharmaceutical manu...",None,"Biotech companies, pharmaceutical companies, r...",None,"400,000,000"
3,7812d7f415357394,JP,Power Diamond Systems,Power Diamond Systems,株式会社Power Diamond Systems,株式会社Power Diamond Systems,株式会社Power Diamond Systems,Power Diamond Systems is a startup conducting ...,Diamond semiconductor devices and modules,Next-generation power electronics including el...,...,power diamond systems,Power Diamond Systems is a startup conducting ...,None,Diamond semiconductor devices and modules,None,Next-generation power electronics including el...,None,None,None,100000000
4,dd11d5fb630ed8f2,JP,Legalscape,Legalscape,株式会社Legalscape,株式会社Legalscape,株式会社Legalscape,株式会社Legalscape provides an AI-powered legal re...,AI legal research platform 'Legalscape' that i...,"Legal professionals including lawyers, corpora...",...,legalscape,株式会社Legalscape provides an AI-powered legal re...,None,AI legal research platform 'Legalscape' that i...,None,"Legal professionals including lawyers, corpora...",None,"Lawyers, corporate legal departments, social i...",None,"160,000,000"
5,e77464f75872db2c,Indonesia,bythen,bythen,bythen,None,bythen,bythen is an AI technology startup specializin...,"Personalized AI assistant, 3D avatar 'Bytes', ...","Global online content creators and consumers, ...",...,bythen,bythen is an AI technology startup specializin...,None,"Personalized AI assistant, 3D avatar 'Bytes', ...",None,"Global online content creators and consumers, ...",None,"Content creators, consumers interested in virt...",None,None
6,edf81f95b08df804,JP,Sukidayo,Sukidayo,None,株式会社すきだよ,株式会社すきだよ,株式会社すきだよ is a startup focused on solving socia...,"カップルTechアプリ「ふたり会議」, a communication app that h...",Couples and families in Japan and potentially ...,...,すきだよ,株式会社すきだよ is a startup focused on solving socia...,None,"カップルTechアプリ「ふたり会議」, a communication app that h...",None,Couples and families in Japan and potentially ...,None,"Couples, spouses, families, and organizations ...",None,None
7,c263a03fa987f706,JP,Legal Agent,Legal Agent,Legal Agent,None,Legal Agent,Legal Agent provides legal services leveraging...,Legal services powered by generative AI and la...,"Enterprises, venture capital (VC) and corporat...",...,legal agent,Legal Agent provides legal services leveraging...,None,Lega